In [ ]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from nltk.tokenize import sent_tokenize
import tokenizers
import transformers
import tensorflow as tf
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm

In [ ]:
pip install pytorch_pretrained_bert

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [ ]:
DATASETDIR1 = 'berttokenizer-base-uncased'
tokenizer = transformers.BertTokenizer.from_pretrained(f'/kaggle/input/{DATASETDIR1}/vocab.txt')

DATASETDIR = 'bert-base-uncased'
config = transformers.BertConfig.from_pretrained(f'/kaggle/input/{DATASETDIR}/bert_config.json')
model = transformers.BertModel.from_pretrained(f'/kaggle/input/{DATASETDIR}/pytorch_model.bin', config=config)

In [ ]:
model_save_name = 'BERT_for_negative_extraction.pt'
path = F"/kaggle/input/bert-negative/{model_save_name}"
model_negative = transformers.BertForTokenClassification.from_pretrained(f'/kaggle/input/{DATASETDIR}/pytorch_model.bin', config=config)
model_negative = model_negative.cuda()
model_negative.load_state_dict(torch.load(path))

In [ ]:
model_save_name = 'BERT_for_neutral_extraction.pt'
path = F"/kaggle/input/bert-neutral/{model_save_name}"
model_neutral = transformers.BertForTokenClassification.from_pretrained(f'/kaggle/input/{DATASETDIR}/pytorch_model.bin', config=config)
model_neutral = model_negative.cuda()
model_neutral.load_state_dict(torch.load(path))

In [ ]:
model_save_name = 'BERT_for_positive_extraction.pt'
path = F"/kaggle/input/bert-positive/{model_save_name}"
model_positive = transformers.BertForTokenClassification.from_pretrained(f'/kaggle/input/{DATASETDIR}/pytorch_model.bin', config=config)
model_positive = model_positive.cuda()
model_positive.load_state_dict(torch.load(path))

In [ ]:
train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
type(model_positive)

In [ ]:
def keywordextract(model, sentence):
    predicted_phrase = []
    extracted_phrase = ""
    text = sentence
    tkns = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
    segments_ids = [0] * len(tkns)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
#     print(tokens_tensor)
#     print(segments_tensors)
    model.eval()
    prediction = []
    logit = model(tokens_tensor, token_type_ids=None,
                                  attention_mask=segments_tensors)   
    logit = logit[0][0].detach().cpu().numpy()    
    
    for p in np.argmax(logit,axis=1):
        prediction.append(p)

    for k, j in enumerate(prediction):
        if j == 1:
            predicted_phrase.append(tokenizer.convert_ids_to_tokens(tokens_tensor[0].to('cpu').numpy())[k])
    for element in predicted_phrase: 
        if ("#" in element):
            element = element.replace("#","")
        extracted_phrase += element
        extracted_phrase += " "
    return extracted_phrase

In [ ]:
test['extracted_phrase'] = ""
for i in range(test.shape[0]):
    if (test['sentiment'][i] == "positive"):
        test['extracted_phrase'][i] = keywordextract(model_positive,test['text'][i])
    if (test['sentiment'][i] == "negative"):
        test['extracted_phrase'][i] = keywordextract(model_negative,test['text'][i])
    if (test['sentiment'][i] == "neutral"):
        test['extracted_phrase'][i] = keywordextract(model_neutral,test['text'][i])

In [ ]:
submission = test[['textID','extracted_phrase']]
submission.columns = ['textID','selected_text']

In [ ]:
submission.to_csv("submission.csv",index=False)

In [ ]:
df = pd.read_csv('/kaggle/working/submission.csv')
df.iloc[:10,:]